In [16]:
import numpy as np
import pandas as pd

In [6]:
datasets_list = pd.read_excel('../data/custom/studies_internal.xlsx')

In [2]:
hela_p = pd.read_excel('../data/processed/proteomes/hela_proteome.xlsx')

In [4]:
hela_p = hela_p[hela_p['is_reviewed'] == 'reviewed']

In [7]:
datasets_hela = {i: pd.read_excel(f'../data/prepared/hela/{i}_int.xlsx') for i in datasets_list.loc[datasets_list['object'] == 'hela', 'study'].to_list()}

for i in datasets_hela:
    datasets_hela[i].iloc[:, -1] = datasets_hela[i].iloc[:, -1].fillna(0)
    datasets_hela[i] = datasets_hela[i][datasets_hela[i].iloc[:, -1] != 0.0] 
    
datasets_hela['wisniewski2012']['copies_per_cell'] = datasets_hela['wisniewski2012']['copies_per_cell'].astype(str)
datasets_hela['wisniewski2012'] = datasets_hela['wisniewski2012'][~datasets_hela['wisniewski2012']['copies_per_cell'].str.contains('<')]
datasets_hela['wisniewski2012']['copies_per_cell'] = datasets_hela['wisniewski2012']['copies_per_cell'].astype(float)

In [23]:
def only_reviewed_ids_1(x):
    if ';' in x:
        x = pd.Series(x.split(';'))
        x = x[x.isin(hela_p['uniprot_id'])]
        x = ';'.join(x.values)
    return x

def only_reviewed_ids_2(x):
    if ';' in x:
        x = pd.Series(x.split(';'))
        x = x.apply(lambda y: y.split('-')[0])
        x = x[x.isin(hela_p['uniprot_id'])]
        x = ';'.join(x.values)
    return x

In [19]:
nap1 = []
acp1 = []

for i in datasets_hela:
    datasets_hela[i]['reviewed_id'] = datasets_hela[i]['uniprot_id'].apply(only_reviewed_ids_1)
    
    datasets_hela[i]['is_group'] = False
    
    datasets_hela[i]['assigned_id'] = np.nan
    datasets_hela[i]['assigned_id_origin'] = np.nan
    
    correct_ids = datasets_hela[i]['reviewed_id'].isin(hela_p['uniprot_id'])
        
    datasets_hela[i].loc[correct_ids, 'assigned_id'] = datasets_hela[i]['reviewed_id']
    datasets_hela[i].loc[correct_ids, 'assigned_id_origin'] = 'reviewed_id'
    
    datasets_hela[i]['is_group'] = datasets_hela[i]['reviewed_id'].str.contains('[\;\ \^\_]')
    datasets_hela[i].loc[datasets_hela[i]['is_group'], 'assigned_id'] = np.nan
    datasets_hela[i].loc[datasets_hela[i]['is_group'], 'assigned_id_origin'] = np.nan
    
    datasets_hela[i]['reviewed_id'] = datasets_hela[i]['reviewed_id'].replace('', np.nan)
    
    nap1.append(len(datasets_hela[i][~datasets_hela[i]['assigned_id'].isna()]) / len(datasets_hela[i]))
    acp1.append(datasets_hela[i][~datasets_hela[i]['assigned_id'].isna()]['copies_per_cell'].sum() / datasets_hela[i]['copies_per_cell'].sum())

In [24]:
nap2 = []
acp2 = []

for i in datasets_hela:
    datasets_hela[i]['reviewed_id'] = datasets_hela[i]['uniprot_id'].apply(only_reviewed_ids_2)
    
    datasets_hela[i]['is_group'] = False
    
    datasets_hela[i]['assigned_id'] = np.nan
    datasets_hela[i]['assigned_id_origin'] = np.nan
    
    correct_ids = datasets_hela[i]['reviewed_id'].isin(hela_p['uniprot_id'])
        
    datasets_hela[i].loc[correct_ids, 'assigned_id'] = datasets_hela[i]['reviewed_id']
    datasets_hela[i].loc[correct_ids, 'assigned_id_origin'] = 'reviewed_id'
    
    datasets_hela[i]['is_group'] = datasets_hela[i]['reviewed_id'].str.contains('[\;\ \^\_]')
    datasets_hela[i].loc[datasets_hela[i]['is_group'], 'assigned_id'] = np.nan
    datasets_hela[i].loc[datasets_hela[i]['is_group'], 'assigned_id_origin'] = np.nan
    
    datasets_hela[i]['reviewed_id'] = datasets_hela[i]['reviewed_id'].replace('', np.nan)
    
    nap2.append(len(datasets_hela[i][~datasets_hela[i]['assigned_id'].isna()]) / len(datasets_hela[i]))
    acp2.append(datasets_hela[i][~datasets_hela[i]['assigned_id'].isna()]['copies_per_cell'].sum() / datasets_hela[i]['copies_per_cell'].sum())

In [31]:
list(zip(acp1, acp2))

[(0.931516265375218, 0.931516265375218),
 (0.9208675494947153, 0.7112249057014499),
 (0.9213065811139222, 0.7054204491571046),
 (0.8879753705498975, 0.6549495727308384),
 (0.9463782036438582, 0.9463782036438582),
 (0.9146951504951742, 0.6722369940568914),
 (0.9267741424654976, 0.9267741424654976)]